# `Материалы кафедры ММП факультета ВМК МГУ. Введение в глубокое обучение.`

# `Лекция 08. Трансформеры`

##### `Материалы составил Алексеев Илья (@voorhs)`

#### `Москва, Весенний семестр 2025`

## `Вспоминаем RNN`

На прошлой лекции вы узнали про принципы работы RNN:
- обучаемые преобразования для обработки последовательностей в рекуррентном режиме 
- shared parameters на каждом рекуррентном шаге
- контекстуализация векторных представлений слов
- механизм внимания в автокодировщике как дополнительный механизм памяти помимо рекуррентного вектора контекста

Мы вводили рекуррентную ячейку:

![rnn-cell](./figures/rnn-rnn-cell.drawio.svg)

Она имеет два входа $h,x$ и два выхода $h',y$:
$$
h'=g_h(W_1x+W_2h),\quad
y=g_y(W_3h'),
$$

Одним из применений была задача seq2seq моделирования с помощью автокодировщика:

![rnn-seq2seq](./figures/rnn-translation.drawio.svg)

Давайте запишем псевдокод алгоритма работы кодировщика и декодировщика




### Алгоритм "RNN Encoder"

---

ВХОД:

- $E=[e(w_1), \ldots, e(w_T)]\in\mathbb{R}^{T\times d}$

ВЫХОД:

- $Y=[y_1,\ldots, y_T]\in\mathbb{R}^{T\times d}$
- $H=[h_1,\ldots, h_T]\in\mathbb{R}^{T\times d}$

---

1. $h_0 := 0$.
2. **Для** $ t $ от 1 до $ T $:
   - $ h_t, y_t := \text{RNNCell}(h_{t-1}, e(w_t)) $.

---



### Алгоритм "RNN Decoder" (в режиме обучения)

---

ВХОД:

- $\tilde E=[e(\text{BOS}), e(\tilde w_1), \ldots, e(\tilde w_{n-1})]\in\mathbb{R}^{n\times d}$
- $h_T\in\mathbb{R}^d$

ВЫХОД:

- $Z=[z_0, z_1, \ldots, z_{n-1}]\in\mathbb{R}^{n\times d}$

---

1. $s_{-1} := h_T$.
2. $e(\tilde w_0):=e(\text{BOS})$
3. **Для** $ t $ от 0 до $ n -1$:
   - $ s_t, z_t := \text{RNNCell}(s_{t-1}, e(\tilde w_{t})) $.

---



### `Механизм внимания`

![](./figures/attention.drawio.svg)

$$
s_t=g_s(W_1z_t+W_2s_{t-1}+W_3c_t),\quad c_t=\sum_{i=1}^Th_i\alpha_{it},\\
\alpha_{it}={\exp(\text{sim}(s_{t-1},h_i))\over\sum_{j=1}^T\exp(\text{sim}(s_{t-1},h_j))},\quad\text{sim}(x,y)=\langle x,y\rangle.
$$




### Алгоритм "RNN Decoder с механизмом внимания"


**ВХОД:**  
- $\tilde E=[e(\text{BOS}), e(\tilde w_1), \ldots, e(\tilde w_{n-1})\in\mathbb{R}^{n\times d}$  
- $H=[h_1, h_2, \ldots, h_T]\in\mathbb{R}^{T\times d}$ 
- $Y=[y_1,\ldots,y_T]\in\mathbb{R}^{T\times d}$

**ВЫХОД:**  

- $Z=[z_0, z_1, \ldots, z_{n-1}]\in\mathbb{R}^{n\times d}$  

---

1. $s_{-1} := h_T$.
2. $e(\tilde w_0):=e(\text{BOS})$

3. **Для** $ t $ от 0 до $ n -1$:  

   - **Для** $ i $ от 1 до $ T $:  
   $$
   \alpha_{it} := \frac{\exp(\text{sim}(s_{t-1}, h_i))}{\sum_{j=1}^T \exp(\text{sim}(s_{t-1}, h_j))}
   $$
   - $c_t := \sum_{i=1}^T h_i \alpha_{it}$
   - $s_t, z_t := \text{RNNCell}(s_{t-1}, e(\tilde w_{t}), c_t).$

---

На этой лекции мы разовьем идею механизма внимания таким образом, что пройдем путь от RNN до трансформера — архитектуры, в которой все рекуррентные связи заменены механизмом внимания.



## `Механизм внимания в векторной форме`

Проанализируем эти формулы с точки зрения векторных операций. Алгоритм подсчёта атеншена для одного шага следующий:

---
ВХОД:
- $q\in\mathbb{R}^d$  [запрос, например $s_t$]
- $K\in\mathbb{R}^{T\times d}$ [ключи, например $h_1,\ldots,h_T$]
- $V\in\mathbb{R}^{T\times d}$ [значения, например $y_1,\ldots,y_T$]
---
ВЫХОД:
- $c\in\mathbb{R}^d$ [собранный контекст, например $c_t$]

---

1. $m := Kq$
2. $\alpha:=\text{Softmax}(m)$
3. $c:=V^T\alpha$

---

Операцию, задаваемую этим алгоритмом, обозначим $\text{Attention}(q,K,V)$. Алгоритм декодировщика можно записать так:

---

1. $s_{-1} := h_T$.
2. $e(\tilde w_0):=e(\text{BOS})$
3. **Для** $ t $ от 0 до $ n -1$:  
    - $c_t := \text{Attention}(s_{t-1}, H, H)$
    - $s_t, z_t := \text{RNNCell}(s_{t-1}, e(\tilde w_{t}), c_t).$

---

Обратимся к псевдокоду и схеме RNN с атеншеном. Мы видим, что в этом алгоритме есть два способа прокидывания связей между словами: рекуррентные формулы и атеншен пулинг. Идея механизма самовнимания в том, чтобы убрать рекуррентные связи и заменить их атеншен пулингом. Сделаем это аккуратно, распишем отдельно для кодировщика и для декодировщика.



### `Самовнимание в кодировщике`

Попытаемся сначала придумать наивный алгоритм, как можно заменить рекуррентные связи на механизм внимания.

---

ВХОД:

- $E=[e(w_1), \ldots, e(w_T)]\in\mathbb{R}^{T\times d}$

ВЫХОД:

- $Y=[y_1,\ldots, y_T]\in\mathbb{R}^{T\times d}$
- $H=[h_1,\ldots, h_T]\in\mathbb{R}^{T\times d}$

---

1. **Для** $ t $ от 1 до $ T $:
    - $ c_t := \text{Attention}(e(w_t), E, E)$
    - $ y_t := \text{FC}(c_t) $.

---

В данном случае мы заменили $\text{RNNCell}$ на связку $\text{Attention}+\text{FC}$.

Мы успешно заменили рекуррентные связи в кодировщике. Принципиальное достоинство этого алгоритма в следующем:
- каждое слово в последовательности (кодировщика или декодировщика) имеет непосредственную связь со всеми другими словами в этой последовательности (а не как было в RNN: связь с непосредственными соседями).
- Т.е. последовательность "смотрит" на саму себя, отсюда название алгоритма

Далее, ключевая идея в том, чтобы обнаружить пространство для оптимизации: заметьте, что все итерации цикла можно посчитать независимо. В функцию $\text{Attention}$ можно подать в качестве $q$ не один вектор $e(w_t)$, а сразу всю последовательность $E$. В алгоритме атеншена изменится лишь то, что вместо умножения $Kq\in\mathbb{R}^{T}$ будет умножение $QK^T\in\mathbb{R}^{T\times T}$.

---
ВХОД:
- $Q\in\mathbb{R}^{T\times d}$
- $K\in\mathbb{R}^{T\times d}$
- $V\in\mathbb{R}^{T\times d}$
---
ВЫХОД:
- $C\in\mathbb{R}^{T\times d}$

---

1. $m := QK^T/\sqrt{d}$
2. $A:=\text{Softmax}(m)$  [построчно]
3. $C:=A V$

---

Если записывать коротко:
$$
\text{Attention}(Q,K,V)=\text{Softmax}(QK^T)V.
$$
Полный алгоритм кодировщика тогда следующий:

---

ВХОД:

- $E=[e(w_1), \ldots, e(w_T)]\in\mathbb{R}^{T\times d}$

ВЫХОД:

- $Y=[y_1,\ldots, y_T]\in\mathbb{R}^{T\times d}$

---

1. $ C := \text{Attention}(E, E, E)$
2. $ Y := \text{FC}(C) $.

---

![](./figures/encoder-block-naive.drawio.svg)



### `Самовнимание в декодировщике`

Как сделать все то же самое в декодировщике? Наивно, это можно сделать так:

---

**ВХОД:**  
- $ \tilde E=[e(\text{BOS}), e(\tilde w_1), \ldots, e(\tilde w_{n-1}) ]\in\mathbb{R}^{n\times d}$  
- $H=[h_1, h_2, \ldots, h_T]\in\mathbb{R}^{T\times d}$ 
- $Y=[y_1,\ldots,y_T]\in\mathbb{R}^{T\times d}$

**ВЫХОД:**  

- $Z=[z_0, z_1, \ldots, z_{n-1}]\in\mathbb{R}^{n\times d}$  

---

1. $e(\tilde w_0):=e(\text{BOS})$
1. **Для** $ t $ от 0 до $ n -1$: 
    - $s_t := \text{Attention}(e(\tilde w_t), \tilde E, \tilde E)$ [самовнимание]
    - $c_t := \text{Attention}(s_t, H, Y)$ [кросс-внимание]
    - $z_t := \text{FC}(c_t)$.

---

Первое, что мы можем улучшить, это добавить параллелизм, поскольку итерации цикла независимы:

---

**ВХОД:**  
- $ \tilde E=[e(\text{BOS}), e(\tilde w_1), \ldots, e(\tilde w_{n-1}) ]\in\mathbb{R}^{n\times d}$  
- $H=[h_1, h_2, \ldots, h_T]\in\mathbb{R}^{T\times d}$ 
- $Y=[y_1,\ldots,y_T]\in\mathbb{R}^{T\times d}$

**ВЫХОД:**  
- $ Z=[z_0, z_1, \ldots, z_{n-1} ]\in\mathbb{R}^{n\times d}$  

---


1. $S := \text{Attention}(\tilde E, \tilde E, \tilde E)$
2. $C := \text{Attention}(S, H, Y)$
3. $Z := \text{FC}(C)$.

---

Отлично! Вы помните, что помимо самовнимания в декодировщике ещё используется внимание, обращённое к выходам кодировщика. В контексте трансформеров это внимание называют кросс-вниманием, т.е. одна последовательность "смотрит" на другую.

![](figures/decoder-block-naive.drawio.svg)

Правда этот алгоритм ничему полезному не обучится. Надо вспомнить, что декодировщик учится на задачу Causal Language Modeling. В строке 1. самовнимание работает на всю последовательность и в обе стороны, прямо как в кодировщике. Это означает, что в активации $z_t$, по которой мы планирует делать предсказание токена $\tilde w_{t+1}$ уже будет содержаться информация о $e(\tilde w_{t+1})$. Происходит утечка меток.

Назовем следующий алгоритм операцией $\text{MaskedAttention}(Q,K,V)$

---

ВХОД:
- $Q\in\mathbb{R}^{T\times d}$
- $K\in\mathbb{R}^{T\times d}$
- $V\in\mathbb{R}^{T\times d}$
---
ВЫХОД:
- $C\in\mathbb{R}^{T\times d}$

---

1. $m := QK^T/\sqrt{d}$
2. $m := \text{triu}(d, -\infty)$ [верхняя треугольная матрица $d\times d$ со значениями $-\infty$]
3. $A:=\text{Softmax}(m)$  [построчно]
4. $C:=A V$

---




### `Обучаемое внимание`

В оригинальном атеншене для рекуррентных сетей были обучаемые параметры. Они были зашиты в функции $\text{sim}(x,y)=\text{FC}(x,y)$ — полносвязной сети которая принимает на вход конкатенацию $x$ и $y$. Это делает операцию атеншен пулинга обучаемой, что добавляет гибкости и репрезентативной мощности архитектуре в целом. В введённом нами алгоритме используется $\text{sim}(x,y)=\langle x,y\rangle$. Как можно было бы ввести обучаемые параметры в нее?

Введем линейные преобразованием перед операцией атеншена:
$$
\text{SelfAttention}(X)=\text{Attention}(XW_q,XW_k,XW_v),\\
\text{CrossAttention}(X,Y)=\text{Attention}(XW_q,YW_k,YW_v),
$$
где $W_q,W_k,W_v\in\mathbb{R}^{d\times d}$.



## `Нормализация и skip-connection`

![](./figures/encoder-block.drawio.svg)

---
ВХОД:
- $X=[x_1,\ldots,x_T]\in\mathbb{R}^{T\times d}$
---
ВЫХОД:
- $Y=[y_1,\ldots,y_T]\in\mathbb{R}^{T\times d}$
---
1. $C:=\text{LayerNorm}(X+\text{SelfAttention}(X))$
4. $Y:=\text{LayerNorm}(C+\text{FC}(C))$
---

У декодировщика ситуация аналогичная

![](./figures/decoder-block.drawio.svg)

---
ВХОД:
- $\tilde X=[\tilde x_1,\ldots,\tilde x_T]\in\mathbb{R}^{T\times d}$
- $Y=[y_1,\ldots,y_T]\in\mathbb{R}^{T\times d}$
---
ВЫХОД:
- $\tilde Y=[\tilde y_1,\ldots,\tilde y_T]\in\mathbb{R}^{T\times d}$
---
1. $S:=\text{LayerNorm}(\tilde X + \text{MaskedSelfAttention}(\tilde X))$
1. $S:=\text{LayerNorm}(S + \text{CrossAttention}(S, Y))$
5. $\tilde Y:=\text{LayerNorm}(S+\text{FC}(S))$
---




## `Вычислительная сложность трансформера `

Если взглянуть на псевдокод RNN и посчитать число операций и потребление памяти, то получится $O(T)$, где $T$ — длина последовательности. У трансформера в механизме внимания возникает перемножение матриц $QK^T$, поэтому сложность трансформера и потребление памяти $O(T^2)$.

Как так? Мы получили что новая архитектура поедает колоссально много ресурсов, особенно в случае экстра длинных последовательностей. Однако может показаться парадоксом, но за одно и то же время трансформер способен пройти большее число шагов оптимизации, чем RNN. Дело в том, что в алгоритме RNN присутсвует цикл по $t$, т.е. алгоритм принципиально последовательный, а такое медленно считается на GPU. В трансформере же все сведено к матричному перемножению --- эта операция эффективно параллелизуется на GPU и позволяет утилизировать больше вычислительной мощности. Т.е. трансформеру действительно нужно сделать больше операций с вещественными числами, но эти операции намного быстрее.

Это объясняется архитектурными особенностями GPU:

- SIMD: single instruction multiple data
- Иерархия памяти: read/write и cache missing
- High-throughput, а не low-latency

<img src="./figures/gpu-memory-hirerarchy.png" style="zoom:33%;" />

Однако существует много архитектурных приемов ускорения трансформеров. Почти все они направлены на эффективное вычисление и хранение матриц внимания.



## `Многоголовочность`

Когда говорят про трансформеры, почти всегда подразумевают использование трюка с использованием многоголовочности атеншена.

$$
\text{head}_i=\text{Attention}(Q\underbrace{W_q}_{d\times {d\over n}},K\underbrace{W_k}_{d\times {d\over n}},V\underbrace{W_v}_{d\times {d\over n}})\in\mathbb{R}^{T\times {d\over n}}, \quad i=\overline{1,n},\\
\text{MultiHeadAttention}(Q,K,V)=\text{concat}(\text{head}_1,\ldots,\text{head}_n)\underbrace{W_o}_{d\times d}
$$

Благодаря этому механизму возникает два эффекта:
- ансамблирование
- узкое горлышко
- еще больше параллелизма



## `Полносвязный блок`

Блок position wise feed forward обладает одинаковой структурой почти во всех трансформерах: это двухслойная сеть, которая из активации размерности $d$ получает активацию размерности $4d$ и затем обратно $d$. 

![](./figures/ff.drawio.svg)
$$
\text{FC}(X):=g(X\underbrace{W_1}_{d\times {4d}}+b_1)\underbrace{W_2}_{4d\times d}+b_2.
$$

Смысл такой архитектуры двоякий:
- Мягкий словарь. Эти блоки составляют подавляющую часть всех параметров трансформера. Логично предположить что в них хранится большая часть знаний трансформера как языковой модели о внешнем мире.
- Утилизация GPU. Такие сети называют широкими. Мы можем противопоставить ей глубокую сеть: что если взять полносвязную сеть с тем же числом параметров, но сделать много узких слоев вместо двух широких? Эта архитектура плоха по той же причине, почему плох RNN: лучше использовать одну большую но параллелизуемую операцию, чем много маленьких но последовательных.

В оригинальном трансформере и в целом в трансформерах долгое время использовались функции активации наподобие ReLU и GELU. Сейчас все чаще используются SwiGLU (особенно в LLM):

1. $h_1:=XW_1+b_1$ [расширение $d\to 4d$]
2. $h_2:=XW_2+b_2$ [расширение $d\to 4d$]
3. $h_\text{SwiGLU}:=h_1\circ\sigma(h_1)\circ h_2$ [рамерность $4d$]
4. $\text{FC}(X):=h_\text{SwiGLU}W_3+b_3$ [сужение $4d\to d$]



## `Позиционная кодировка`

Заметим одну примечательную особенность: атеншен обрабатывает множество, а не последовательность. В рекуррентных сетях порядок слов учитывался в самом алгоритме за счёт использования цикла по t. Сейчас же мы имеем только перемножение матриц, порядок слов отражается только в порядке строк в этих матрицах, но не в самих признаках. В текущем виде информация о порядке слов нигде не учитывается.

Трансформеры в базовом варианте используют прием под названием позиционная кодировка. Идея в том, чтобы ко входным эмбедингам слов добавлять специальный эмбединг, соответствующий номеру слова в предложении. Оригинальный трансформер использовал фиксированные эмбединги, получаемые с помощью синусов и косинусов. Затем сообщество открыло, что эти эмбединги можно просто инициализировать случайными и обучать. Сейчас продвинутые трансформеры вообще отошли от этой идеи и используют методы, которые напрямую влияют на механизм подсчёта атеншена.

Синусоидная кодировка состоит в том, чтобы завести матрицу $P\in\mathbb{R}^{T\times d}$, где $T$ — максимальная длина последовательностей, которые наша модель будет поддерживать. Матрица определяется следующими правилами:
$$
P_{m,2i}:=\sin\left({m\over10000^{2i/d}}\right),\quad
P_{m,2i+1}:=\cos\left({m\over10000^{2i/d}}\right),
$$
для всех $i=\overline{1,d/2}$.



## `Полная схема трансформера`

<img src="./notes.assets/transformer.jpg" style="zoom: 33%;" />
